In [10]:
#!pip install ~/cloudfiles/code/software/PyPeanuts/latest/peanuts-0.6.5-py3-none-any.whl --force-reinstall

In [11]:
import pandas as pd
import peanuts
from peanuts.AML.orion import *

import tensorflow as tf
print(tf.version.VERSION) # shuold be 2.3.1
from model import EfficientGAN

username='746220'
orion = Orion(user=username)

#import umap
#import umap.plot
#import matplotlib.pyplot as plt

2.3.1
🔔 Just a Reminder: Is your key vault name following the naming convention: ba-<n|p>-z<eaus|weus>-<user>-kv? If NO, please specify your key vault name.
🔔 Just a Reminder: Do you have a secret with name corpaaid-pw stored in your key vault? If NO, please specify your secret name.


In [12]:
def sql_path_loader(data_source, scoring_training_ind):
    """
    This function loads query paths based on input args.
    We have saved queries in these locations.
    """
    
    if data_source == 'trans':
      if scoring_training_ind=='training':
        sql_path="../AWD_TRANS_Databricks_Query.sql"  # _Feb24-1.txt"
      else:
        # sql_path="/dbfs/FileStore/tables/data_query_test_Mar4.txt"
        sql_path="../AWD_TRANS_Databricks_Query.sql"
#       data_query_updated_Feb-1.txt"
      #data_query_Jan-3.txt" data_query_Jan_vol.txt"  data_query_Jan_vol_cast.txt
#     elif data_source == "clickstream":
#             sql_path = "./"
    return sql_path

# sql_path_loader('trans')

def sql_renderer(data_source, scoring_training_ind, start_date, end_date, print_sql=False):
    """
    This function renders the query based on input args
    """
    
    sql_path = sql_path_loader(data_source = data_source, scoring_training_ind=scoring_training_ind )
    sql = open(sql_path, 'r').read()
#     if scoring_training_ind=='training':
    sql = sql.format(lb=start_date, ub=end_date)
#     else:
#       sql = sql.format(CURRENT_DATE=date)
    if print_sql:
        print(sql)
    return sql
  
def data_retriever_fun(data_source,scoring_training_ind, start_date, end_date, print_sql):
    """
    This function retrieves records from mosaic tables
    """
        
    if (data_source != "trans") & (data_source != "clickstream"):
        print("### ERROR: data source should be either trans or clickstream ###")
        return
    elif (start_date == None):
        print("### ERROR: Provide date in format of 'yyyy-mm-dd' ###")
        return
    else:
            sql = sql_renderer(data_source = data_source, scoring_training_ind=scoring_training_ind, 
            start_date=start_date, end_date=end_date, print_sql = print_sql)
            retrieved_df = orion.mq(sql)#pd.read_sql(sql)
            return retrieved_df

In [13]:
#sql_path="AWD_TRANS_Databricks_Query.sql"
#sql = open(sql_path, 'r').read()
#start_date='2022-03-24'
#end_date='2022-03-30'
#sql = sql.format(lb=start_date, ub=end_date, binHist='(1)')


In [14]:
sdf_test=data_retriever_fun(data_source='trans', scoring_training_ind='scoring', start_date='2022-03-30', end_date='2022-04-05', print_sql=False)
sdf=data_retriever_fun(data_source='trans', scoring_training_ind='training', start_date='2022-02-28', end_date='2022-03-29', print_sql=False)

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy


In [ ]:
# sdf.display()
# without filter
indices=['LYLTY_ACCT_ID', 'PNR_LOCTR_ID', 'PNR_CREATE_DT', 'AWD_PKG_ISSUE_DT']
categorical=['OPERAT_AIRLN','ORIGIN_RM_WRLD_REGION_CD','DESTNTN_RM_WRLD_REGION_CD']
#, 'OPERAT_AIRLN_IATA_CD2', 'OD_ORIGIN_AIRPRT_IATA_CD','OD_DESTNTN_AIRPRT_IATA_CD','ORIGIN_CNTRY_CD','ORIGIN_RM_WRLD_REGION_CD','DESTNTN_CNTRY_CD','DESTNTN_RM_WRLD_REGION_CD'
#  'VISITED_BEFORE_STATE'
#'AWD_LEVEL_MILE_QTY','NUM_PAX','SAFE_AIRPRT_IND','FLAGGED_AIRPRT_IND', 
numericals=['AP','PAX_LYLTY_IND', 
            'VISITED_BEFORE_AIRPORT',
            #'VISITED_BEFORE_CNTRY',
            'USED_BEFORE_ARILN',
            'ONE_WAY', 'SHORT_AP_OK',
            'SEEN_ITIN',
            #'SAFE_VS_FLAGGED_CNTRY', 
            'BIN_HIST', 'MULTI_REDEEM', 'EMAIL_CHANGE30', 
            'U_AP', 'AGE45', 'AGE_logEMAIL',
            'LAST_EMAIL',
            'AGE_LAST_EMAIL',
            #'HOME_AIRPRT',
            #'HOME_STATE',
            #'HOME_RESDNC_CNTRY',
            #'HOME_ADV',
            'FLAGGED_CNTRY_ALERT',
            'FLAGGED_AIRPRT_ALERT',
            'risk_feature1', 
            'risk_feature2',
            'risk_feature3',
            'risk_feature4',
            #'risk_feature5',
            'risk_feature6',
 #           'safe_feature1',
 #           'FREQ_USED_ARILN',
 #           'FREQ_DESTNTN_CNTRY',
 #           'FREQ_ORIGIN_CNTRY',
 #           'FREQ_CNTRY',
#            'FREQ_ORIGIN_AIRPRT',
#            'FREQ_DESTNTN_AIRPRT',
 #           'FREQ_AIRPORT',
           ]

# 'FLAGGED_CNTRY_ALERT', 'SAFE_CNTRY_OK',
#  'SAFE_VS_FLAGGED_CNTRY',
#  'NUM_HIST',
#  'DAYS_SINCE_LAST'
predictors=numericals+categorical

In [ ]:
#sdfNorm = sdf.query('(BIN_HIST == 1) & (NORMAL_TRAN == 1)')
sdfNorm = sdf.query('(BIN_HIST == 1) & ((NORMAL_TRAN == 1) | (FLOWN_LAST_NM == 1) | (RED_LAST_NM == 1) | (IP_MATCH == 1))')

sdfNorm = sdfNorm.drop_duplicates()
#sdfFiltered = sdf.filter((sdf.OWNER_IN_FLIGHT != 1) & (sdf.BIN_HIST == 1))

#sdf_test_clean = sdf_test.query('((BIN_HIST == 1) & (RAW_AP <= 9) & ~(NORMAL_TRAN == 1))')
sdf_test_clean = sdf_test.query('((BIN_HIST == 1) & (RAW_AP <= 9) & ~((NORMAL_TRAN == 1) | (FLOWN_LAST_NM == 1) | (RED_LAST_NM == 1) | (IP_MATCH == 1)))')
sdf_test_clean = sdf_test_clean.drop_duplicates()
#predictions=model.transform(sdf_test_clean)

#& sdf.OWNER_IN_FLIGHT != 1 & sdf.PAX_REDEEMED_BEFORE != 1 & sdf.ONE_WAY != 1)

numAfter = len(sdfNorm)
numBefore = len(sdf)
print(f"rows of normal instances {numAfter}")
print(f"rows of all instances {numBefore}")
print(f"reduction percentage AFTER filtering {round(numAfter / numBefore * 100, 2)}")

print(f"rows AFTER filtering {len(sdf_test_clean)}")
print(f"rows BEFORE filtering {len(sdf_test)}")

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import LabelEncoder# creating initial dataframe
import pandas as pd

df = sdfNorm
dfTest = sdf_test_clean

le = LabelEncoder()
dfCopy = df.copy()
dfTestCopy = dfTest.copy()

for col in categorical:
  dfCopy[col] = le.fit_transform(dfCopy[col])
  dfTestCopy[col] = le.fit_transform(dfTestCopy[col])

In [ ]:
model = EfficientGAN()
model.fit(dfCopy[predictors])
proba = model.predict(dfTestCopy[predictors])
dfTest['decision_function'] = proba
out = dfTest.nsmallest(n=100, columns='decision_function')

In [ ]:
out.to_csv('res220330-0405.csv')

In [20]:
#import umap
#import umap.plot
#import matplotlib.pyplot as plt

#mapper = umap.UMAP(random_state=42).fit(dfTestCopy[predictors])
#umap.plot.points(mapper)

In [ ]:
#clf = IsolationForest(n_estimators=100, max_samples=4096, random_state=0).fit(dfCopy[predictors])

#dd = pd.DataFrame(clf.decision_function(dfTestCopy[predictors]))
#dfTest['decision_function'] = dd
#out = dfTest.nsmallest(n=100, columns='decision_function')

In [ ]:
import sys
print(sys.version)